In [678]:
import pandas as pd
import numpy as np
import myfitnesspal
import datetime
import json

from flask import g, Flask, render_template, request, session, redirect, url_for, jsonify
from werkzeug.wrappers import Request, Response

from werkzeug.serving import run_simple

# MyFitnessPal

In [1147]:
client = myfitnesspal.Client('groupb3', password='groupb3thebest')

In [1148]:
date = datetime.date.today()
day = client.get_date(date)
print(day)
print(day.exercises[0].get_as_list())

calories = 0
#calories = day.exercises[0].get_as_list()[0]['nutrition_information']['calories burned']

print('You burned ' + str(calories) + ' calories today')

03/24/22 {}
[]
You burned 0 calories today


In [1149]:
current_weight = client.get_measurements(upper_bound = datetime.date.today())[datetime.date(2022, 3, 1)]
print('Your current weight is: ' + str(current_weight) + 'kg')

Your current weight is: 85.0kg


In [1182]:
calories = 0

# WebApp

In [1183]:
## Stores (prefer get from DB ofcourse later on)
#Create list or dict from multiple stores
#52.351476166967736, 4.93562849866492
stores = [{'lat' : 52.356, 'lng' : 4.953, 'name' : "Science park", 'description' : 'Campus of Science Park'},
         {'lat' : 52.351476166967736, 'lng' : 4.93562849866492, 'name' : "De Meer", 'description' : 'Snackbar at the middenweg'}]
# lat = 52.356
# lng = 4.953
# loc = "Science park"
# content_string = 'DESCRIPTION OF THIS STORE'

In [1184]:
## Hotspots
#Create list or dict for multiple hotspots
52.353580934267285, 4.955375250247711
hotspots = [{'lat' : 52.353580934267285, 'lng' : 4.955375250247711, 'name' : "Annas Tuin en Ruigte", 'description' : 'Een onverwacht groen paradijsje midden op het Science Park.'},
         {'lat' : 52.349141869498425, 'lng' : 4.944225460426125, 'name' : "Jaap Eden Ijsbaan", 'description' : 'Lekker schaatsen, erg koud wel'}]

#hotspots = [{'lat' : 52.353580934267285, 'lng' : 4.955375250247711}]
lat = 52.353580934267285
lng = 4.955375250247711
lat2 = 52.349141869498425
lng2 = 4.944225460426125
loc = "Science park"

content_string = 'DESCRIPTION OF THIS HOTSPOT'

# { lat: {{lat}}, lng: {{lng}} }
# [{ location: {'lat' : 52.353580934267285, 'lng' : 4.4.955375250247711}}]

In [1185]:
app = Flask(__name__)
app.secret_key = "abc"  


In [1186]:
# @app.route("/")
# def index(name=None):
#     return render_template('map.html', lng = lng, lat = lat, content_string = content_string)

@app.route("/", methods=['POST', 'GET'])
def index(name=None):
    if request.method == 'POST':
        session['distance'] = 0
        #print(request.form['storelist'])
        if request.form['storelist']:
            cur_store = request.form['storelist']
            print(cur_store)
            json_acceptable_string = cur_store.replace("'", "\"")
            dict_store = json.loads(json_acceptable_string)
            if calories < 300:
                return render_template('direction_scenic.html', store_list = dict_store, hotspots = hotspots, lng = lng, lat = lat, lng2 = lng2, lat2 = lat2, content_string = content_string, loc = loc)
            if calories > 300:
                return render_template('direction_short.html', store_list = dict_store, hotspots = hotspots, lng = lng, lat = lat, lng2 = lng2, lat2 = lat2, content_string = content_string, loc = loc)
#         elif request.form['storelist1']:
#             cur_store = request.form['storelist1']
#             print(cur_store)
#             json_acceptable_string = cur_store.replace("'", "\"")
#             dict_store = json.loads(json_acceptable_string)
#             return render_template('direction_short.html', store_list = dict_store, hotspots = hotspots, lng = lng, lat = lat, lng2 = lng2, lat2 = lat2, content_string = content_string, loc = loc)
    else:
        return render_template('index.html', stores = stores, name = name)

In [1187]:
@app.route("/map")
def maps():
    return render_template('map.html', lng = lng, lat = lat, content_string = content_string)

In [1188]:
@app.route("/direction_scenic")
def direction_scenic():
    session['distance'] = 0
    return render_template('direction_scenic.html', lng = lng, lat = lat, hotspots = hotspots, content_string = content_string, loc = loc)

In [1189]:
@app.route("/direction_short")
def direction_short():
    return render_template('direction_short.html', lng = lng, lat = lat, content_string = content_string, loc = loc)

In [1190]:
@app.route("/profile", methods = ['POST', 'GET'])
def profile(calories = calories, current_weight = current_weight):
    user_name = request.form.get('exampleInputEmail1')
    print(session['distance'])
    return render_template('profile.html', calories = calories, current_weight = current_weight)

In [1191]:
@app.route("/finish_page", methods=['POST', 'GET'])
def finish_page():
    if request.method == 'POST':
        if request.form['save']:
            print(request.form['save'])
            print('doet het')
            new_stats = request.form['save']
            session['distance'] = new_stats
            return render_template('profile.html', calories = calories, current_weight = current_weight, new_stats = new_stats)
    else:
        distance = session.get('distance')
        print(distance)
        return render_template('finish_page.html', distance = distance, lng = lng, lat = lat, content_string = content_string, loc = loc)

In [1192]:
run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Mar/2022 11:28:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2022 11:28:17] "GET /static/images/icon.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Mar/2022 11:28:17] "GET /static/images/profile.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Mar/2022 11:28:17] "GET /static/images/maps.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Mar/2022 11:28:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2022 11:28:18] "GET /static/images/icon.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Mar/2022 11:28:18] "GET /static/images/profile.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Mar/2022 11:28:18] "GET /static/icon.png HTTP/1.1" 404 -
127.0.0.1 - - [24/Mar/2022 11:28:18] "GET /static/icon.png HTTP/1.1" 404 -


{'lat': 52.356, 'lng': 4.953, 'name': 'Science park', 'description': 'Campus of Science Park'}


127.0.0.1 - - [24/Mar/2022 11:28:23] "GET /finish_page HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2022 11:28:23] "GET /static/images/icon.png HTTP/1.1" 304 -
127.0.0.1 - - [24/Mar/2022 11:28:23] "GET /static/images/profile.png HTTP/1.1" 304 -


0
